Задание:

С помощью метода get_prepared_all_vacancies_from_hh
загрузить 10000 вакансии с сайта hh.ru.

Договоритесь чтобы вакансии не повторялись.

Сохранить результат в виде csv файла с тремя столбцами:
1. id вакансии
2. сырой json
3. подготовленное описание вакансии

Не забывайте про обработку ошибок

In [89]:
import pickle
import nltk
import pymystem3
from urllib.request import urlopen
import json
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


def download_json(url):
    response = urlopen(url)
    data_json = json.loads(response.read())
    print(data_json['error'])
    return data_json

# пример: download_json('https://api.hh.ru/vacancies/43551857')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [85]:
'''
РЕАЛИЗОВАТЬ remove_garbage
Используйте регулярные выражения
IN: str '<p>ТРЕБОВАНИЯ: <br /> - обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
'''
import re


def remove_garbage(raw_text):
    text = raw_text
    text = re.sub("<[^>]*>", "", text)
    text = re.sub("&quot;", "", text)
    return text

In [3]:
from nltk.tokenize import word_tokenize

'''
РЕАЛИЗОВАТЬ tokenize
Используйте word_tokenize из nltk.tokenize
IN: str 'ТРЕБОВАНИЯ: - обязательно высшее или незаконченное высшее экономическое образование;...'
OUT: list ['требования', ':', '-', 'обязательно', 'высшее', 'или', 'незаконченное', 'высшее', 'экономическое', 'образование']
'''


def tokenize(raw_text):
    tokens = word_tokenize(raw_text, language="russian")
    return tokens

In [4]:
from spacy import load
from nltk.stem import SnowballStemmer

'''
РЕАЛИЗОВАТЬ to_base_form
Можно использовать и лемматизацию и стемминг: pymystem3, ntlk, Spacy, gensim
IN: str 'Красивая мама красиво мыла раму'
OUT: str 'красивый мама красиво мыть рама'
'''


def to_base_form(raw_text):
    tokens = []
    lemmer = SnowballStemmer("russian")
    # lemmer = load('ru_core_news_md', disable='tok2vec')
    for word in raw_text:
        token = lemmer.stem(word)
        tokens.append(token)
    return tokens

In [5]:
from nltk.corpus import stopwords

'''
РЕАЛИЗОВАТЬ remove_stop_words
Используйте stopwords из nltk.corpus 
IN: list ['требования', ':', '-', 'обязательно', 'высший', 'или', 'незаконченный', 'высший', 'экономический', 'образование']
OUT: list ['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование']
'''


def remove_stop_words(raw_words):
    stop_words = set(stopwords.words('russian'))
    prep_signs = {",", ".", ":", ";", "-", "_", "\'", "\"", "/", "\\", "+", "=", "(", ")"}
    stop_words = stop_words.union(prep_signs)
    filtered_words = []
    for word in raw_words:
        if word not in stop_words:
            filtered_words.append(word)
    return filtered_words

In [77]:
from nltk.tokenize import sent_tokenize

'''
РЕАЛИЗОВАТЬ get_prepared_vacancy_description_from_hh
Используйте реализованные ранее методы
IN: int 72323
OUT1: str vacancy_json dump
OUT2: list[list] [['требование', 'обязательно', 'высший', 'незаконченный', 'высший', 'экономический', 'образование', ...], [...], ...]
'''


def get_prepared_vacancy_from_hh(vacancy_id):
    base_url = 'https://api.hh.ru/vacancies/'
    #try:
    vacancy_json = download_json(base_url + str(vacancy_id))
    #except HTTPError as e:
    #raise Exception(e.code)
    vacancy_description = vacancy_json['description']  # получить описание вакансии из json
    vacancy_description_sentences = sent_tokenize(vacancy_description, "russian")
    prepared_vacancy_description = []
    for sentence in vacancy_description_sentences:
        sentence = remove_garbage(sentence)
        description = tokenize(sentence)
        description = to_base_form(description)
        description = remove_stop_words(description)
        #print(description)
        for desc in description:
            prepared_vacancy_description.append(desc)
    return vacancy_json, prepared_vacancy_description

In [87]:
from urllib.error import HTTPError

'''
РЕАЛИЗОВАТЬ get_prepared_all_vacancies_from_hh
Используйте get_prepared_vacancy_from_hh и ThreadPoolExecutor из concurrent.futures
IN: list [int]
OUT: list [vacancy_id, raw_description, prepared_description]
'''
from concurrent.futures import ThreadPoolExecutor


def get_prepared_all_vacancies_from_hh(vacancy_ids):
    executor = ThreadPoolExecutor(max_workers=16)
    future_list = []
    for vacancy_id in vacancy_ids:
        try:
            future = executor.submit(get_prepared_vacancy_from_hh, vacancy_id)
            future_list.append(future)
        except URLError:
            continue
        except TimeoutError:
            continue
        except Exception:
           continue

    result = []
    for future in future_list:
        future_result = future.result()
        result.append(future_result)
    return result

In [95]:
from urllib.error import HTTPError, URLError

#try:
#pass
#with open('data.pickle', 'rb') as f:
#s = pickle.load(f)
#except FileNotFoundError:
#s = []
s = []
try:
    vacancy_ids = range(100, 301)
    s.append(get_prepared_all_vacancies_from_hh(vacancy_ids))
except Exception as e:
    # print(e)
    pass
with open('data.pickle', 'wb') as f:
    pickle.dump(s, f)
s

[]

In [11]:
s

[]